In [15]:
# Import necessary libraries
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from api_credentials import client_id, client_secret, redirect_uri
from playlist_membership_check import get_tracks_positions_in_playlists
import json

# Load playlists from JSON file
with open('playlists.json', 'r') as file:
    playlists_dict = json.load(file)

# Initialize the Spotify client with client credentials for public data access
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


## Example usage of `get_tracks_position_in_playlist`

Checking which playlists and positions Noah Kahan's 'Stick Season' is in

In [16]:
# Example: Checking playlists and position for Noah Kahan 'Stick Season'
# using it's track_id
track_ids = [
    #'0mflMxspEfB0VbI1kyLiAv', 
    '2lJn77IeZAP9cmv6DPXOrL',
]

track_positions = get_tracks_positions_in_playlists(client_id, client_secret, redirect_uri, playlists_dict, track_ids)

for track_id, playlists in track_positions.items():
    print(f"Track ID {track_id} is in the following playlists at these positions:")
    for playlist_name, positions in playlists.items():
        positions_str = ", ".join(str(position + 1) for position in positions)  # Create a string of positions, accounting for 0 indexing
        print(f"Playlist: {playlist_name}: Position #{positions_str}")
    if not playlists:
        print("This track is not in any of the given playlists.")

Track ID 2lJn77IeZAP9cmv6DPXOrL is in the following playlists at these positions:
Playlist: New Music Friday AU & NZ: Position #13
Playlist: Fresh Country: Position #4
Playlist: Pop n' Fresh: Position #20
Playlist: Singled Out: Position #3
